## Imports

In [237]:
import processing_utils
import graphing_utils

In [238]:
# import from processing_utils
import importlib
importlib.reload(processing_utils)
from processing_utils import ERP_epoching

In [248]:
# import from graphing_utils
importlib.reload(graphing_utils)
from graphing_utils import (load_filter_file, large_bin_markers, small_bin_markers, lines_cleared, helper_line_config, calibration_calc, plot_graph)

## Load Data

In [249]:
# Data Info
user = ""
sessionID = ""

# XDF file name and folder path
f_name = f"filt_{user}_{sessionID}"
exp_type = ""
data_sub_dir = ""
file_type = "xdf"

# Location of filtered XDF file
xdf_file_loc = f"sample/{exp_type}/{data_sub_dir}/{user}/{sessionID}/{f_name}.{file_type}"

# Location to save plots
figure_loc = f'sample/{exp_type}/{data_sub_dir}/{user}/{sessionID}/test_plots/'

sfreq = 250

In [250]:
# load data
EEG, markers, calibration_markers = load_filter_file(xdf_file_loc)

## Markers | Events

Stream starts inside calibration
*calibration_markers*
**102**: End of Calibration
**103**: Start of Eyes Closed Period
**104**: End of Eyes Closed Period
**105**: Start of Tutorial
**106**: End of Tutorial

*markers*
**0**: Start of Game
**1**: End of Game
**2**: Row Cleared
**4**: Game Knows to Send Helper (Crossed Threshold, spawn_help event)
**5**: Game Sends Helper (see_help event)
**6**: Helper Clears Rows (help event)
**7-**: Levels

In [251]:
# Check Markers
print("calibration markers")
print(calibration_markers[calibration_markers != 500])

print("\nmarkers")
print(markers[markers != 500])

calibration markers
[102 103 104 105 106]

markers
[ 0  7  8  9 10 11 12 13 14 15  2  2 16  2 17 18  2  2 19  2  2 20 21  2
  2 22  2  2 23 24  2 25  2  2 26 27  2  2  2 28 29  4  2 30 31  1]


## Large Time Bin Processing + Plotting

In [252]:
# size of large time bin, in seconds
large_bin_size = 10

# configure markers
large_markers, time_idx_large, result_large = large_bin_markers(markers, large_bin_size, int(sessionID))

# get lines cleared in each time bin
result_large = lines_cleared(markers, time_idx_large, result_large)

# get helper related event line locations in plot
result_large = helper_line_config(result_large, markers, int(sessionID))

# epoch EEG into each time bin
epochs_large = ERP_epoching(EEG, large_markers, result_large['targets'], t_range=large_bin_size, sfreq=sfreq, ignore=sfreq*3)

In [ ]:
# plotting params
band = 'Alpha'
chan_picks = ['Fp1', 'Fp2']
freq_list = [(8,13)]
color = 'bisque'

# params for scipy.signal.welch function
# nperseg: window size
nperseg = 250
# noverlap: number of samples to overlap between windows
noverlap = 0

# calculate baselines
result_large = calibration_calc(result_large, EEG, calibration_markers, freq_list, sfreq=sfreq)

save = True

plot_graph(epochs_large, result_large, band, chan_picks, freq_list, color, nperseg=nperseg, noverlap=noverlap, sfreq=sfreq, fname=f_name, filepath=figure_loc, save=save)

## Small Time Bin Processing + Plotting

In [255]:
# min_time to start looking at, in seconds (must be <= 0)
min_time = -10
# max_time to look at, in seconds (must be >= 0)
max_time = 10
# size of small time bin, in seconds
small_bin_size = 2
# see small_bin_markers doc string for more information

# configure markers
small_markers, time_idx_small, result_small = small_bin_markers(markers, min_time, max_time, small_bin_size, int(sessionID))

# get lines cleared in each time bin
result_small = lines_cleared(markers, time_idx_small, result_small)

# get helper related event locations in plot
result_small = helper_line_config(result_small, markers, int(sessionID), sfreq=sfreq)

# epoch data into small time bins
epochs_small = ERP_epoching(EEG, small_markers, result_small['targets'], t_range=small_bin_size, sfreq=sfreq, ignore=sfreq*3)

[91767, 92267, 92767, 93267, 93767, 94267, 94767, 95267, 95767, 96267]


In [ ]:
# plotting params
band = 'Alpha'
chan_picks = ['Fp1', 'Fp2']
freq_list = [(8,13)]
color = 'bisque'

# params for scipy.signal.welch function
# nperseg: window size
nperseg = 250
# noverlap: number of samples to overlap between windows
noverlap = 0

# calculate baselines
result_small = calibration_calc(result_small, EEG, calibration_markers, freq_list, sfreq=sfreq)

save = True

plot_graph(epochs_small, result_small, band, chan_picks, freq_list, color, nperseg=nperseg, noverlap=noverlap, sfreq=sfreq, fname=f_name, filepath=figure_loc, save=save)